In [1]:
from pandas import DataFrame, read_csv
from csv import reader
from numpy import mean, array, zeros, errstate, seterr
from collections import defaultdict
from scipy.spatial.distance import cosine
from scipy.stats import spearmanr
from os import path
from gensim.models import KeyedVectors, Word2Vec
from pymorphy2 import MorphAnalyzer

morph = MorphAnalyzer()
seterr(all='raise')

Using Theano backend.


{'divide': 'ignore', 'invalid': 'ignore', 'over': 'ignore', 'under': 'ignore'}

Load raw messy corpus proposed by the Laboratory of Neurolinguistics and transform it to a decent dataset

In [2]:
with open(path.join('data', 'data.csv'), mode='r') as infile:
    corpus_bytes = reader(infile)
    corpus = [i for i in corpus_bytes]

corpus = [i[0].split('\t') if len(i) == 1 else ''.join(i).split('\t') for i in corpus]
df = DataFrame(corpus[1:], columns=corpus[0])

Amount of unique words in a corpus

In [3]:
len(df['word.id'].unique())

801

Aggregate corpus data by unqiue words

In [4]:
df = df.groupby('Lemma').agg({
                            'average.accuracy': ', '.join,
                            'IA_DWELL_TIME': ', '.join,
                            'IA_FIRST_FIXATION_DURATION': ', '.join,
                            'IA_FIRST_RUN_DWELL_TIME': ', '.join,
                            'IA_FIRST_RUN_FIXATION_COUNT': ', '.join,
                            'IA_FIXATION_COUNT': ', '.join,
                            'IA_LEGAL': ', '.join,
                            'IA_REGRESSION_IN': ', '.join,
                            'IA_REGRESSION_OUT_FULL': ', '.join,
                            'IA_REGRESSION_PATH_DURATION': ', '.join,
                            'IA_SECOND_RUN_DWELL_TIME': ', '.join,
                            'ao': ', '.join,
                            'IA_SELECTIVE_REGRESSION_PATH_DURATION': ', '.join,
                            'IA_SKIP': ', '.join,
                            'IA_SPILLOVER': ', '.join,
                            'landing': ', '.join,
                            'dir': ', '.join,
                            'fixated.letter': ', '.join,
                            'one_fix': ', '.join,
                            'twoplus_fix': ', '.join
                        }).reset_index()

Serialize transformed dataset

In [5]:
df.to_csv('data_words.csv')

Calculate mean value of aggregated values for each word in the dataset

In [6]:
for column in df:
    for i in range(len(df)):
        try:
            values = list(filter(lambda a: a != 'NA', [a.strip() for a in df[column][i].split(',')]))
            df[column][i] = mean([float(a) for a in values])
        except ValueError: # if column is a column of words
            continue
        except FloatingPointError: # invalid value encountered in double_scalars
            df[column][i] = None
df = df.dropna()

/Users/defeater/anaconda/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2889: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)


Loading and processing dataset of human judgements of embeddings

In [7]:
def load_sim_dataset(name, embeddings, verbose=False):
    df = read_csv(path.join('data', '{}.csv'.format(name))).dropna()
    old_len = len(df)
    for i, m in df.iterrows():
        if not m['word1'] in embeddings or not m['word2'] in embeddings:
            df.drop(i, inplace=True)
    if verbose:
        print('Percent of dropped = {:2.1f}%, amount of remanining words = {}'.format((old_len - len(df))/old_len*100, len(df)))
    return df.reset_index(drop=True)

In [8]:
def make_sims_dataset(dataset, embeddings):
    sims = zeros(shape=len(dataset), dtype='float32')
    for i, m in dataset.iterrows():
        sims[i] = 1 - cosine(embeddings[m['word1']], embeddings[m['word2']])
    return sims

In [9]:
def make_word2vec_dataset(dataset, model):
    sims = zeros(shape=len(dataset), dtype='float32')
    for i, m in dataset.iterrows():
        sims[i] = 1 - cosine(model[add_pos_tag(m['word1'])], model[add_pos_tag(m['word2'])])
    return sims

In [10]:
rusvectores = KeyedVectors.load_word2vec_format(path.join('..', 'models', 'rusvectores.bin'), binary=True)

In [11]:
def add_pos_tag(word):
    tag = str(morph.parse(word)[0].tag.POS)
    if tag == 'ADJF':
        tag = 'ADJ'
    elif tag == 'INFN':
        tag = 'VERB'
    if word == 'объем': # sorry
        tag = 'NOUN'
    if word == 'струя':
        tag = 'NOUN'
    if word == 'чай':
        tag = 'NOUN'
    if word == 'два':
        word = 'двадцать'
        tag = 'NUM'
    return '{}_{}'.format(word, tag)

In [12]:
STARTING_AMOUNT = 20

for columns_amount in range(STARTING_AMOUNT,len(df.columns)):
    eye_embeddings = defaultdict()
    for i, k in df.ix[:,0:columns_amount].iterrows():
        eye_embeddings[k['Lemma']] = array(k[1:].values)
    print('Using {} features of the data'.format(columns_amount))
    for name, printed_name in [
                ('simlex999', 'RuSimLex999'),
                ('simlex965', 'RuSimLex965'),
                ('hj', 'HJ: Human Judgements of Word Pairs'),
                ('rt', 'RT: Synonyms and Hypernyms from the Thesaurus RuThes'), 
                ('ae2', 'AE: Cognitive Associations from the Sociation.org Experiment'),
                ]:
        print('{}'.format(printed_name))
        dataset = load_sim_dataset(name, eye_embeddings, True)
        try:
            eye_sims = make_sims_dataset(dataset, eye_embeddings)
            word2vec_sims = make_word2vec_dataset(dataset, rusvectores)
        except FloatingPointError: # invalid value encountered in double_scalars
            eye_sims = zeros(shape=len(dataset), dtype='float32')
            print('NaN')
            print()
            continue
        print('Correlation with human judgements: {}'.format(spearmanr(eye_sims, dataset.sim)))
        print('Correlation of human judmenets with Word2Vec: {}'.format(spearmanr(word2vec_sims, dataset.sim)))
        print('Correlation With word2Vec: {}'.format(spearmanr(word2vec_sims, eye_sims)))
        print()

/Users/defeater/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  """


Using 20 features of the data
RuSimLex999
Percent of dropped = 97.8%, amount of remanining words = 22
Correlation with human judgements: SpearmanrResult(correlation=0.29072402840403871, pvalue=0.1893225753876453)
Correlation of human judmenets with Word2Vec: SpearmanrResult(correlation=0.20248872017246275, pvalue=0.36613882836713274)
Correlation With word2Vec: SpearmanrResult(correlation=0.28773318258903335, pvalue=0.19412392123350533)

RuSimLex965
Percent of dropped = 97.8%, amount of remanining words = 21
Correlation with human judgements: SpearmanrResult(correlation=0.27633295828034554, pvalue=0.22529758979842845)
Correlation of human judmenets with Word2Vec: SpearmanrResult(correlation=0.27698315347629932, pvalue=0.2241633525290902)
Correlation With word2Vec: SpearmanrResult(correlation=0.38531513970110459, pvalue=0.084537153629326484)

HJ: Human Judgements of Word Pairs
Percent of dropped = 97.2%, amount of remanining words = 11
Correlation with human judgements: SpearmanrResult(c